In [3]:
# Define the finite field
p = 0xc90102faa48f18b5eac1f76bb40a1b9fb0d841712bbe3e5576a7a56976c2baeca47809765283aa078583e1e65172a3fd
F = GF(p)

# Define the elliptic curve parameters
a = F(0xa079db08ea2470350c182487b50f7707dd46a58a1d160ff79297dcc9bfad6cfc96a81c4a97564118a40331fe0fc1327f)  # coefficient a
b = F(0x9f939c02a7bd7fc263a4cce416f4c575f28d0c1315c4f0c282fca6709a5f9f7f9c251c9eede9eb1baa31602167fa5380)  # coefficient b

E = EllipticCurve(F, [a, b])

order = E.order()
print(f"Order of the curve: {order}")
print(order.factor())

True
Order of the curve: 30937339651019945892244794266256713890440922455872051984762505561763526780311616863989511376879697740787911484829297
35809 * 46027 * 56369 * 57301 * 65063 * 111659 * 113111 * 7072010737074051173701300310820071551428959987622994965153676442076542799542912293


In [23]:
print("Generator x", hex(21619587544885632079095357635634910300113008113595371382084832484239997227378229645965723431736571510143715960589749))
print("Generator x", hex(21746213218696329518285360739570925937175688707127725191522900994294195925044227501605308519669493584773867551629937))

Generator x 0x8c771d8d67e1744f2431e8be26ff9adaa81dd5d2a7aa5326b954ac6c3d314bbcf485224f1021f47276b83332ad468db5
Generator x 0x8d49ba41789783466e33f4f86f3745f4817eae2518dd44893a7611ffdcb27590dfc1611819336769255d482fe40ca271


In [5]:
client_gen_x = 1305488802776637960515697387274764814560693662216913070824404729088258519836180992623611650289275235949409735080408
client_gen_y = 2840284555446760004012395483787208388204705580027573689198385753943125520419959469842139003551394700125370894549378
client_gen_p = E(client_gen_x, client_gen_y)
print("Client Gen", client_gen_p, (E.order() == client_gen_p.order()))

client_x = 0x195b46a760ed5a425dadcab37945867056d3e1a50124fffab78651193cea7758d4d590bed4f5f62d4a291270f1dcf499
client_y = 0x357731edebf0745d081033a668b58aaa51fa0b4fc02cd64c7e8668a016f0ec1317fcac24d8ec9f3e75167077561e2a15
client_p = E(client_x, client_y)
print("Client Point", client_p, (E.order() == client_p.order()))

server_x = 0xb3e5f89f04d49834de312110ae05f0649b3f0bbe2987304fc4ec2f46d6f036f1a897807c4e693e0bb5cd9ac8a8005f06
server_y = 0x85944d98396918741316cd0109929cb706af0cca1eaf378219c5286bdc21e979210390573e3047645e1969bdbcb667eb
server_p = E(server_x, server_y)
print("Server Point", server_p, (E.order() == server_p.order()))

Client Gen (1305488802776637960515697387274764814560693662216913070824404729088258519836180992623611650289275235949409735080408 : 2840284555446760004012395483787208388204705580027573689198385753943125520419959469842139003551394700125370894549378 : 1) True
Client Point (3902729749136290727021456713077352817203141198354795319199240365158569738643238197536678384836705278431794896368793 : 8229109857867260486993831343979405488668387983876094644791511977475828392446562276759399366591204626781463052691989 : 1) True
Server Point (27688886377906486650974531457404629460190402224453915053124314392088359043897605198852944594715826578852025617899270 : 20559737347380095279889465811846526151405412593746438076456912255094261907312918087801679069004409625818172174526443 : 1) True


In [9]:
factors = factor(E.order())

partial_results = {}
for p, e in list(factors)[:-1]:
        pe = p^e
        Pp = (order // pe) * client_gen_p
        Qp = (order // pe) * client_p
        
        if Pp.is_zero():
            continue
            
        x_p = 0
        p_power = 1
        
        for i in range(e):
            P_curr = (p^(e-1-i)) * Pp
            Q_curr = Qp - x_p * Pp
            digit = discrete_log(Q_curr, P_curr, p, operation='+')
            x_p += digit * p_power
            p_power *= p
            
        partial_results[p] = x_p

print(partial_results)

{35809: 11872, 46027: 42485, 56369: 12334, 57301: 45941, 65063: 27946, 111659: 43080, 113111: 57712}


In [28]:
factors = factor(order)
partial_solution = 0
if partial_results:
    moduli = [p^e for p, e in factors[:-1]]
    values = [partial_results[p] for p, e in factors[:-1]]
    partial_solution = CRT(values, moduli)

factors = [x[0] for x in list(factor(E.order()))]

for k in range(factors[-1]):
    final_result = partial_solution + k * prod(factors[:-1])
    if (final_result * client_gen_p == client_p):
        print(final_result)
        break 

168606034648973740214207039875253762473


In [36]:
privkey = 168606034648973740214207039875253762473

shared_key = server_p * 168606034648973740214207039875253762473
print(shared_key)

(9285933189458587360370996409965684516994278319709076885861327850062567211786910941012004843231232528920376385508032 : 380692327439186423832217831462830789200626503899948375582964334293932372864029888872966411054442434800585116270210 : 1)
